In [1]:
from src.first_model.preprocessing import PreProcessingFirstModel
from src.first_model.train import Modeling

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import math

from sklearn.feature_extraction.text import CountVectorizer

[nltk_data] Downloading package stopwords to /home/luba/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/luba/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
PATH_RAW = "/home/luba/Documents/DS/projects-courses-ongoing/bncc-classifier-[doing]/data/raw/bncc_first_classifier.csv"  # noqa
PATH_TO_SAVE = "/home/luba/Documents/DS/projects-courses-ongoing/bncc-classifier-[doing]/data/curated/bncc_first_classifier_cleaned.csv"  # noqa

In [4]:
lista1 = ["a", "b"]
str(lista1)

"['a', 'b']"

In [3]:
prepro_obj = PreProcessingFirstModel(PATH_RAW)
frq_words, rare_words = prepro_obj.prepro()

# export list of frq and rare words on our data
# with open("frq_words.txt", "w") as f:
#     # print(frq_words)
#     _ = [f.write("%s\n" % item) for item in frq_words]

# with open("rare_words.txt", "w") as f:
#     # print(rare_words)
#     _ = [f.write("%s\n" % item) for item in rare_words]

# prepro_obj.export_cleaned_data(PATH_TO_SAVE)
modeling = Modeling(prepro_obj.data)
# # modeling.tokenization()
modeling.split_data()
# modeling.saving_pipeline()
# X_train_trans, X_test_trans = modeling.make_bag_of_words()
# modeling.train_evaluate_log_reg(
#     X_train_trans=X_train_trans, X_test_trans=X_test_trans
# )

In [4]:
modeling.X_train

,questions,target,questions_clean
77438,<p>Verificando-se a massa corporal de cinco cr...,Matemática,verificandose massa corporal cinco crianças re...
8211,"<p>Como o <span style=""color:rgb(65, 65, 65);f...",Educação Física,sumô chegou brasil
5218,"<p style=""text-align:justify;"">Contudo, o ser ...",Arte,contudo ser humano produz coisas apesar terem ...
27150,"<p>(ENEM) - 2010</p><p><br></p><p style=""paddi...",Inglês,enem 2010the weather manthey say british love ...
69111,"<p><span style=""color:rgb(0, 0, 0);font-family...",Matemática,arnaldo planeja ir praia deseja utilizar camis...
...,...,...,...
27265,<p>Question Edited</p><p>Considerando as discu...,Física,question editedconsiderando discussões atuais ...
28356,<p>Sobre as micoses que acometem os seres huma...,Biologia,sobre micoses acometem seres humanos assinale ...
97160,<p>Leia o trecho a seguir.</p><p><br></p><p><e...,História,leia trecho seguirpolítica valorização café co...
66343,<p>Como se l&ecirc; a 4<sup>3</sup>?</p>,Matemática,lê 43


## Baseline BOW

In [5]:
# baselines
models = {
    "rg_lg": LogisticRegression(max_iter=500, n_jobs=8),
    "r_forest": RandomForestClassifier(max_depth=500, n_jobs=8),
    "lgbm": LGBMClassifier(n_jobs=8),
}

In [6]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer()
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.7448963626874732
Model: r_forest. Macro avg F1: 0.6574655622325649
Model: lgbm. Macro avg F1: 0.729495005234485
Model: rg_lg. Macro avg F1: 0.7449595142009064


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {"C": [0.01, 0.1, 1, 10, 100]}

rcv = GridSearchCV(
    LogisticRegression(),
    param_grid=param_grid,
    cv=3,
    scoring="roc_auc",
    verbose=True,
    n_jobs=8,
)

count_vectorizer = CountVectorizer(ngram_range=(2, 2))
count_vectorizer.fit(x_train)
X_train_cv = count_vectorizer.transform(x_train)
X_val_cv = count_vectorizer.transform(x_val)

rcv.fit(X_train_vect, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:   39.8s finished


GridSearchCV(cv=3, estimator=LogisticRegression(), n_jobs=8,
             param_grid={'C': [0.01, 0.1, 1, 10, 100]}, scoring='roc_auc',
             verbose=True)

## Baseline N-Grams 2, 3

### Baseline N-grams only 2

In [34]:
from sklearn.model_selection import GridSearchCV

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        param_grid = {"C": [0.01, 0.1, 1, 10, 100]}

        rcv = GridSearchCV(
            LogisticRegression(),
            param_grid=param_grid,
            cv=3,
            scoring="f1_macro",
            verbose=True,
            n_jobs=8,
        )
        count_vectorizer = CountVectorizer(ngram_range=(2, 2))
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        # cv_classifier = model
        rcv.fit(X_train_cv, y_train)
        y_pred = rcv.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


: 

: 

### Baseline N-grams with 1 and 2 grams

In [7]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )

        count_vectorizer = CountVectorizer(ngram_range=(1, 2))
        count_vectorizer.fit(x_train)
        X_train_cv = count_vectorizer.transform(x_train)
        X_val_cv = count_vectorizer.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

Model: rg_lg. Macro avg F1: 0.7393553737736352
Model: r_forest. Macro avg F1: 0.6520310106809519
Model: lgbm. Macro avg F1: 0.7294101706242744


KeyboardInterrupt: 

## Word2Vec

In [10]:
from gensim.models import Word2Vec

In [8]:
modeling.X_train

,questions,target,questions_clean
77438,<p>Verificando-se a massa corporal de cinco cr...,Matemática,verificandose massa corporal cinco crianças re...
8211,"<p>Como o <span style=""color:rgb(65, 65, 65);f...",Educação Física,sumô chegou brasil
5218,"<p style=""text-align:justify;"">Contudo, o ser ...",Arte,contudo ser humano produz coisas apesar terem ...
27150,"<p>(ENEM) - 2010</p><p><br></p><p style=""paddi...",Inglês,enem 2010the weather manthey say british love ...
69111,"<p><span style=""color:rgb(0, 0, 0);font-family...",Matemática,arnaldo planeja ir praia deseja utilizar camis...
...,...,...,...
27265,<p>Question Edited</p><p>Considerando as discu...,Física,question editedconsiderando discussões atuais ...
28356,<p>Sobre as micoses que acometem os seres huma...,Biologia,sobre micoses acometem seres humanos assinale ...
97160,<p>Leia o trecho a seguir.</p><p><br></p><p><e...,História,leia trecho seguirpolítica valorização café co...
66343,<p>Como se l&ecirc; a 4<sup>3</sup>?</p>,Matemática,lê 43


In [16]:
wv = Word2Vec(
    sentences=modeling.X_train["questions_clean"].apply(lambda x: x.split()),
    vector_size=100,
    window=3,
    min_count=1,
    workers=10,
)

In [17]:
def transforma_palavra(question):
    # Define dicionário para abrigar os vetores de palavras
    dict_vetores = {}
    # Usando a função get_vector, transforma a palavra no vetor de Embeddings
    for palavra in question.split():
        dict_vetores[palavra] = wv.wv.get_vector(palavra)
    return dict_vetores

In [18]:
transforma_palavra(modeling.X_train["questions_clean"].values[11])

{'possível': array([-0.6336333 ,  0.68504906, -0.31369463,  0.20733002, -0.14555071,
        -1.8603587 , -1.7340254 ,  2.9368505 , -0.55471766, -0.0246266 ,
        -1.0871179 , -1.6103001 , -1.0372006 ,  1.6274679 ,  1.3726625 ,
         1.2210501 ,  1.397302  , -4.178638  ,  2.3975673 , -1.8942193 ,
         2.0105045 , -0.09053704,  1.7372606 , -1.9379561 , -1.3401536 ,
         1.2166493 , -0.14307518, -1.5084463 ,  0.23437762, -1.861257  ,
         3.194247  , -1.2691952 , -0.34585598, -0.27976665,  0.6264136 ,
        -0.10887279,  2.539996  ,  0.16884762, -2.1291804 , -1.2661866 ,
         0.130509  , -1.9170808 ,  0.37218472, -0.8761023 ,  0.7154536 ,
        -1.3059803 , -2.4497354 , -0.02353578, -0.45040086,  2.1176026 ,
         0.28525132,  1.513944  ,  0.16542171,  2.44571   , -0.51265323,
         1.0204229 ,  0.9114708 ,  0.45969144, -1.1418794 , -0.17017783,
        -0.8179298 ,  0.27603024, -0.41462952,  1.5848671 , -1.0208013 ,
         0.3921023 ,  1.2518939 ,  0.48

In [20]:
wv.wv.most_similar("observar")

[('perceber', 0.9443067908287048),
 ('identificar', 0.9384897351264954),
 ('notar', 0.922267496585846),
 ('encontrar', 0.9194545149803162),
 ('relacionar', 0.9173203706741333),
 ('reconhecer', 0.9159761071205139),
 ('associar', 0.9152820110321045),
 ('verificar', 0.9148088693618774),
 ('visualizar', 0.9048494100570679),
 ('classificar', 0.8972480297088623)]

## Embeddings

In [22]:
import numpy as np


def transforma_tweet(question):
    # Usando a função get_vector, transforma a palavra no vetor de Embeddings
    lista_vetores = [wv.wv.get_vector(x) for x in question.split()]
    return np.sum(lista_vetores, axis=0)

In [23]:
import pandas as pd

# Aplicando sobre os nossos tweets
vetores_embeddings = modeling.X_train["questions_clean"].apply(transforma_tweet)

# Criando um Dataframe com os resultados
df_embeddings = pd.DataFrame.from_dict(
    dict(zip(vetores_embeddings.index, vetores_embeddings.values))
).T
# Definindo os nomes das colunas
df_embeddings.columns = ["embedding_" + str(i) for i in range(1, 101)]

# Vamos também trazer o tweet original e o sentimento
df_embeddings["question"] = modeling.X_train["questions_clean"]
df_embeddings["sentimento_tweet"] = modeling.X_train["target"].values

df_embeddings.head()

,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,...,embedding_93,embedding_94,embedding_95,embedding_96,embedding_97,embedding_98,embedding_99,embedding_100,question,sentimento_tweet
77438,18.170626,32.807747,9.959334,29.166166,-24.648773,-24.674480,1.709166,37.060497,-32.463932,-9.130833,...,-20.910948,7.648385,31.117918,33.101097,3.353589,-25.041729,2.246368,1.647730,verificandose massa corporal cinco crianças re...,Matemática
8211,3.504960,2.777276,1.473757,1.128671,0.394673,-1.934710,-1.164994,2.748590,-1.012172,-0.512289,...,0.265560,0.166511,0.838237,1.749212,0.910057,-3.654889,2.319661,1.888766,sumô chegou brasil,Educação Física
5218,-26.850327,48.946526,-5.540483,-2.004592,16.228155,-68.135971,-9.607052,81.904266,-52.595112,2.647099,...,2.250904,-5.709748,78.722733,43.318226,17.891273,-51.837765,24.917643,14.833723,contudo ser humano produz coisas apesar terem ...,Arte
27150,-7.067691,7.325456,14.512897,-1.509277,-0.453215,-26.093063,11.365140,30.676512,-6.630595,-8.538989,...,9.457629,7.490003,37.033508,19.222738,25.034550,-15.049734,17.248346,-9.877907,enem 2010the weather manthey say british love ...,Inglês
69111,4.644130,13.412626,3.592278,8.114446,-0.553489,-12.670075,0.811158,17.354336,-7.288770,-4.055807,...,1.854276,2.224144,12.666059,10.407845,0.960609,-5.005013,-1.456441,-6.752326,arnaldo planeja ir praia deseja utilizar camis...,Matemática


In [27]:
from sklearn.model_selection import train_test_split, cross_val_score

# Separando os dados em X e y
X_emb = df_embeddings[df_embeddings.columns[:100]]
y_emb = df_embeddings.sentimento_tweet

X_train_emb, X_test_emb, y_train_emb, y_test_emb = train_test_split(
    X_emb, y_emb, test_size=0.2, stratify=y_emb
)

In [32]:
modelos_teste = {
    "Logística": LogisticRegression(max_iter=500),
    "Árvore": DecisionTreeClassifier(max_depth=10),
    "RandomForest": RandomForestClassifier(max_depth=100),
    "LGBM": LGBMClassifier(),
}

# Treinando cada modelo e avaliando métrica sobre conjuntos de validação
for nome, modelo in modelos_teste.items():
    metrica = cross_val_score(
        modelo, X_train_emb, y_train_emb, cv=3, scoring="f1_macro"
    ).mean()
    print("ROC AUC - " + nome + ": {:.3f}".format(metrica))

/home/luba/.local/share/virtualenvs/bncc-classifier-_doing_-BYzFlin2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/luba/.local/share/virtualenvs/bncc-classifier-_doing_-BYzFlin2/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative

ROC AUC - Logística: 0.565
ROC AUC - Árvore: 0.414
ROC AUC - RandomForest: 0.551
ROC AUC - LGBM: 0.581


## Baseline TFIDF Unigram

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )
        tfidf = TfidfVectorizer()
        tfidf.fit(x_train)
        X_train_cv = tfidf.transform(x_train)
        X_val_cv = tfidf.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        # if name == "gnb":
        #     X_train_cv = X_train_cv.toarray()
        #     X_val_cv = X_val_cv.toarray()

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))

## Baseline TF-IDF Unigrams and Bigrams

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=2020)
for train_idx, val_idx in kfold.split(
    modeling.X_train["questions_clean"], modeling.y_train
):
    for name, model in models.items():
        x_train, y_train = (
            modeling.X_train["questions_clean"].iloc[train_idx],
            modeling.y_train.iloc[train_idx],
        )
        x_val, y_val = (
            modeling.X_train["questions_clean"].iloc[val_idx],
            modeling.y_train.iloc[val_idx],
        )
        tfidf = TfidfVectorizer(ngram_range=(1, 2))
        tfidf.fit(x_train)
        X_train_cv = tfidf.transform(x_train)
        X_val_cv = tfidf.transform(x_val)

        if name == "lgbm":
            X_train_cv = X_train_cv.astype("float32")
            X_val_cv = X_val_cv.astype("float32")
            y_train = y_train.astype("float32")
            y_val = y_val.astype("float32")

        # if name == "gnb":
        #     X_train_cv = X_train_cv.toarray()
        #     X_val_cv = X_val_cv.toarray()

        cv_classifier = model
        cv_classifier.fit(X_train_cv, y_train)
        y_pred = cv_classifier.predict(X_val_cv)
        f1 = f1_score(y_val, y_pred, average="macro")
        # print('ROC AUC - {}: {}'.format(name, mean))
        print("Model: {}. Macro avg F1: {}".format(name, f1))